In [190]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors, rdmolops
import networkx as nx
import lightgbm as lgb
import joblib

import re

In [191]:
# Загрузка данных
df_sorption = pd.read_csv('data/Solvent_Sorption_NCOMMS/ML_uptake_sorption.csv')
df_diffusivity = pd.read_csv('data/Solvent_Diffusivity_NCOMMS/ML_Diffusivity.csv')

# Конфигурация (аналогично Kaggle)
TARGETS = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
useless_cols = [
    # Список из вашего кода (скопируйте полностью)
    'BCUT2D_MWHI', 'BCUT2D_MWLOW', ... # Все перечисленные в коде столбцы
]

# Загрузка обученных моделей (замените пути)
MODELS = {
    'Tg': [joblib.load(f'models/lgb_Tg_fold_{i}.pkl') for i in range(5)],
    'FFV': [joblib.load(f'models/lgb_FFV_fold_{i}.pkl') for i in range(5)],
    'Tc': [joblib.load(f'models/lgb_Tc_fold_{i}.pkl') for i in range(5)],
    'Density': [joblib.load(f'models/lgb_Density_fold_{i}.pkl') for i in range(5)],
    'Rg': [joblib.load(f'models/lgb_Rg_fold_{i}.pkl') for i in range(5)]
}

In [192]:
USELESS_COLS = [
    'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO',
    'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW',
    'NumRadicalElectrons', 'SMR_VSA8', 'SlogP_VSA9', 'fr_barbitur',
    'fr_benzodiazepine', 'fr_dihydropyridine', 'fr_epoxide', 'fr_isothiocyan',
    'fr_lactam', 'fr_nitroso', 'fr_prisulfonamd', 'fr_thiocyan',
    'MaxEStateIndex', 'HeavyAtomMolWt', 'ExactMolWt', 'NumValenceElectrons',
    'Chi0', 'Chi0n', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2n', 'Kappa1',
    'LabuteASA', 'HeavyAtomCount', 'MolMR', 'Chi3n', 'BertzCT', 'Chi2v',
    'Chi4n', 'HallKierAlpha', 'Chi3v', 'Chi4v', 'MinAbsPartialCharge',
    'MinPartialCharge', 'MaxAbsPartialCharge', 'FpDensityMorgan2',
    'FpDensityMorgan3', 'Phi', 'Kappa3', 'fr_nitrile', 'SlogP_VSA6',
    'NumAromaticCarbocycles', 'NumAromaticRings', 'fr_benzene', 'VSA_EState6',
    'NOCount', 'fr_C_O', 'fr_C_O_noCOO', 'NumHDonors', 'fr_amide',
    'fr_Nhpyrrole', 'fr_phenol', 'fr_phenol_noOrthoHbond', 'fr_COO2',
    'fr_halogen', 'fr_diazo', 'fr_nitro_arom', 'fr_phos_ester'
]

In [193]:
def clean_smiles(smiles):
    """
    Улучшенная очистка SMILES с более строгой обработкой:
    1. Предварительная обработка специальных случаев
    2. Замена нестандартных атомов
    3. Валидация и каноникализация
    """
    if not isinstance(smiles, str):
        return np.nan
    
    # Предварительная обработка
    cleaned = smiles.strip()
    
    # Словарь замен для нестандартных атомов и групп
    replacements = {
        '[g]': 'Cl',
        '[t]': 'F',
        '[d]': 'O',  # Меняем на O, так как это часть O[d]
        '[e]': 'O',  # Меняем на O, так как это часть O[e]
        'O[d]': 'OH',  # Заменяем всю группу
        'O[e]': 'OH',  # Заменяем всю группу
    }
    
    # Применяем замены
    for old, new in replacements.items():
        cleaned = cleaned.replace(old, new)
    
    # Дополнительная обработка оставшихся нестандартных конструкций
    cleaned = re.sub(r'\[[a-zA-Z]+\]', 'C', cleaned)  # Заменяем оставшиеся [...] на C
    cleaned = re.sub(r'[^a-zA-Z0-9@=\-\.#*()\[\]%/]', '', cleaned)
    
    # Проверяем валидность и пытаемся канонизировать
    try:
        mol = Chem.MolFromSmiles(cleaned)
        if mol is not None:
            return Chem.MolToSmiles(mol, canonical=True)
        return cleaned
    except:
        return cleaned

In [194]:
def validate_and_fix_smiles(smiles):
    """
    Пытается валидировать и исправить SMILES с несколькими попытками
    """
    if not isinstance(smiles, str):
        return np.nan
    
    # Первая попытка - прямая конверсия
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        return Chem.MolToSmiles(mol, canonical=True)
    
    # Вторая попытка - очистка
    cleaned = clean_smiles(smiles)
    mol = Chem.MolFromSmiles(cleaned)
    if mol is not None:
        return Chem.MolToSmiles(mol, canonical=True)
    
    # Третья попытка - более агрессивная очистка
    aggressive_cleaned = re.sub(r'\[[^\]]+\]', 'C', cleaned)  # Заменяем все [...] на C
    mol = Chem.MolFromSmiles(aggressive_cleaned)
    if mol is not None:
        return Chem.MolToSmiles(mol, canonical=True)
    
    return np.nan

In [195]:
# Функция для канонизации SMILES
def make_smile_canonical(smile):
    try:
        mol = Chem.MolFromSmiles(smile)
        if mol is None:
            return np.nan
        return Chem.MolToSmiles(mol, canonical=True)
    except:
        return np.nan

In [196]:
# Для обработки проблемных SMILES
def compute_graph_features(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return [0, 0, 0]
        
        adj = rdmolops.GetAdjacencyMatrix(mol)
        G = nx.from_numpy_array(adj)
        
        diameter = nx.diameter(G) if nx.is_connected(G) else 0
        avg_path = nx.average_shortest_path_length(G) if nx.is_connected(G) else 0
        num_cycles = len(list(nx.cycle_basis(G)))
        
        return [diameter, avg_path, num_cycles]
    except Exception:
        return [0, 0, 0]  # Значения по умолчанию при ошибках

In [197]:
# Функция для вычисления дескрипторов
def compute_all_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return [np.nan] * len(Descriptors.descList)
    
    return [desc[1](mol) for desc in Descriptors.descList]

In [198]:
# Модифицируем функцию preprocess_smiles
def preprocess_smiles(df, smiles_column='Cleaned_SMILES'):
    """Переработанная функция предобработки"""
    df = df.copy()
    
    # Применяем улучшенную очистку и валидацию
    df['canonical_smiles'] = df[smiles_column].apply(validate_and_fix_smiles)
    
    # Фильтруем только валидные SMILES
    valid_mask = df['canonical_smiles'].notna()
    df_valid = df[valid_mask].copy()
    
    if len(df_valid) == 0:
        print("Предупреждение: Нет валидных SMILES после обработки")
        return df, pd.DataFrame()
    
    # Вычисляем дескрипторы
    desc_names = [desc[0] for desc in Descriptors.descList if desc[0] not in USELESS_COLS]
    descriptors = []
    
    for smi in df_valid['canonical_smiles']:
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            descriptors.append([np.nan] * len(desc_names))
        else:
            desc_values = [desc[1](mol) for desc in Descriptors.descList if desc[0] not in USELESS_COLS]
            descriptors.append(desc_values)
    
    df_descriptors = pd.DataFrame(descriptors, columns=desc_names)
    
    # Вычисление графовых признаков
    graph_features = [compute_graph_features(smi) for smi in df_valid['canonical_smiles']]
    df_graph = pd.DataFrame(graph_features, columns=['graph_diameter', 'avg_shortest_path', 'num_cycles'])
    
    # Объединение признаков
    features = pd.concat([df_descriptors, df_graph], axis=1)
    features.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    return df_valid, features

In [199]:
def predict_polymer_properties(df, smiles_column='Cleaned_SMILES'):
    # Убедимся, что столбец существует
    if smiles_column not in df.columns:
        raise ValueError(f"Столбец {smiles_column} не найден в датафрейме")
    
    # Создаем копию для безопасной работы
    df = df.copy()
    
    # Добавляем временный идентификатор
    df['temp_index'] = range(len(df))
    
    # Предобработка данных
    try:
        df_valid, features = preprocess_smiles(df, smiles_column)
    except Exception as e:
        print(f"Ошибка предобработки: {e}")
        # Возвращаем датафрейм с NaN значениями
        for target in TARGETS:
            df[f'{target}'] = np.nan
        return df
    
    # Если нет валидных SMILES
    if len(df_valid) == 0:
        print("Предупреждение: Нет валидных SMILES для предсказания")
        for target in TARGETS:
            df[f'{target}'] = np.nan
        return df
    
    # Для каждого свойства загружаем список ожидаемых признаков
    FEATURE_NAMES = {}
    for target in TARGETS:
        try:
            FEATURE_NAMES[target] = joblib.load(f'models/feature_names_{target}.pkl')
        except:
            print(f"Предупреждение: Не удалось загрузить feature_names для {target}")
            # Пытаемся получить имена из модели
            try:
                FEATURE_NAMES[target] = MODELS[target][0].feature_names_
            except:
                print(f"Критическая ошибка: Не удалось получить feature_names для {target}")
                FEATURE_NAMES[target] = features.columns.tolist()
    
    # Предсказание для каждого свойства
    for target in TARGETS:
        # Выбираем только нужные для этой модели признаки
        required_features = FEATURE_NAMES[target]
        missing_features = [f for f in required_features if f not in features.columns]
        
        # Добавляем отсутствующие признаки как NaN
        for feat in missing_features:
            features[feat] = np.nan
            
        # Убедимся, что порядок признаков соответствует ожиданиям модели
        X = features[required_features]
        
        predictions = np.zeros(len(df_valid))
        
        # Ансамблевое предсказание
        valid_models = 0
        for model in MODELS[target]:
            try:
                preds = model.predict(X)
                predictions += preds
                valid_models += 1
            except Exception as e:
                print(f"Ошибка предсказания {target} для модели: {e}")
        
        if valid_models > 0:
            predictions /= valid_models
        else:
            predictions = np.full(len(X), np.nan)
            print(f"Критическая ошибка: Все модели для {target} завершились с ошибкой")
        
        df_valid[f'{target}'] = predictions
    
    # Объединение с исходными данными
    result = df.merge(df_valid[['temp_index'] + [f'{t}' for t in TARGETS]], 
                      on='temp_index', 
                      how='left')
    
    return result.drop(columns=['temp_index'])

In [200]:
def extend_datasets():
    df_sorption = pd.read_csv('data/Solvent_Sorption_NCOMMS/ML_uptake_sorption.csv')
    df_diffusivity = pd.read_csv('data/Solvent_Diffusivity_NCOMMS/ML_Diffusivity.csv')
    
    # Добавляем отладочную информацию
    print(f"Исходное количество записей в датасете сорбции: {len(df_sorption)}")
    print(f"Исходное количество записей в датасете диффузии: {len(df_diffusivity)}")
    
    # Предварительная проверка SMILES
    df_sorption['Valid_SMILES'] = df_sorption['Polymer_SMILES'].apply(validate_and_fix_smiles).notna()
    df_diffusivity['Valid_SMILES'] = df_diffusivity['Polymer_SMILES'].apply(validate_and_fix_smiles).notna()
    
    print(f"Количество валидных SMILES в датасете сорбции: {df_sorption['Valid_SMILES'].sum()}")
    print(f"Количество валидных SMILES в датасете диффузии: {df_diffusivity['Valid_SMILES'].sum()}")
    
    # Расширение датасетов
    print("\nОбработка датасета сорбции...")
    df_sorption_extended = predict_polymer_properties(df_sorption, 'Polymer_SMILES')
    
    print("\nОбработка датасета диффузии...")
    df_diffusivity_extended = predict_polymer_properties(df_diffusivity, 'Polymer_SMILES')
    
    # Сохранение результатов
    df_sorption_extended.to_csv('data/Solvent_Sorption_NCOMMS/ML_uptake_sorption_extended.csv', index=False)
    df_diffusivity_extended.to_csv('data/Solvent_Diffusivity_NCOMMS/ML_Diffusivity_extended.csv', index=False)
    print("Расширение датасетов завершено!")

In [201]:
def analyze_smiles_problems(df, smiles_column='Polymer_SMILES', sample_size=1000):
    """Анализирует и показывает примеры проблемных SMILES"""
    problem_count = 0
    sample_problems = []
    
    for idx, row in df.iterrows():
        smiles = row[smiles_column]
        cleaned = clean_smiles(smiles)
        mol = Chem.MolFromSmiles(cleaned)
        
        if mol is None and problem_count < sample_size:
            problem_count += 1
            sample_problems.append({
                'original': smiles,
                'cleaned': cleaned,
                'index': idx
            })
    
    print(f"\nНайдено {problem_count} проблемных SMILES (первые {len(sample_problems)} примеров):")
    for problem in sample_problems:
        print(f"\nИндекс: {problem['index']}")
        print(f"Оригинал: {problem['original']}")
        print(f"Очищенный: {problem['cleaned']}")
    
    return sample_problems

# Запускаем анализ
print("Анализ датасета сорбции:")
problems_sorption = analyze_smiles_problems(df_sorption)

print("\nАнализ датасета диффузии:")
problems_diffusivity = analyze_smiles_problems(df_diffusivity)

Анализ датасета сорбции:

Найдено 0 проблемных SMILES (первые 0 примеров):

Анализ датасета диффузии:

Найдено 0 проблемных SMILES (первые 0 примеров):


In [202]:
# Сохранение имен признаков для каждой модели
for target in TARGETS:
    feature_names = []
    for i, model in enumerate(MODELS[target]):
        try:
            # Для LGBMRegressor
            names = model.booster_.feature_name()
        except:
            # Для других типов моделей
            names = model.feature_names_in_
        
        feature_names.append(names)
    
    # Берем первый набор (все фолды должны иметь одинаковые признаки)
    joblib.dump(feature_names[0], f'models/feature_names_{target}.pkl')

In [203]:
# Запуск процесса
extend_datasets()

Исходное количество записей в датасете сорбции: 2275
Исходное количество записей в датасете диффузии: 2045


[16:23:12] SMILES Parse Error: syntax error while parsing: CC6(C)CC4(CC(C)(C)c3cc2oc1c(C#N)c([g])c([t])c(C#N)c1oc2cc34)c5cc(O[d])c(O[e])cc56
[16:23:12] SMILES Parse Error: check for mistakes around position 36:
[16:23:12] (C)c3cc2oc1c(C#N)c([g])c([t])c(C#N)c1oc2c
[16:23:12] ~~~~~~~~~~~~~~~~~~~~^
[16:23:12] SMILES Parse Error: Failed parsing SMILES 'CC6(C)CC4(CC(C)(C)c3cc2oc1c(C#N)c([g])c([t])c(C#N)c1oc2cc34)c5cc(O[d])c(O[e])cc56' for input: 'CC6(C)CC4(CC(C)(C)c3cc2oc1c(C#N)c([g])c([t])c(C#N)c1oc2cc34)c5cc(O[d])c(O[e])cc56'
[16:23:12] SMILES Parse Error: syntax error while parsing: CC6(C)CC4(CC(C)(C)c3cc2oc1c(C#N)c([g])c([t])c(C#N)c1oc2cc34)c5cc(O[d])c(O[e])cc56
[16:23:12] SMILES Parse Error: check for mistakes around position 36:
[16:23:12] (C)c3cc2oc1c(C#N)c([g])c([t])c(C#N)c1oc2c
[16:23:12] ~~~~~~~~~~~~~~~~~~~~^
[16:23:12] SMILES Parse Error: Failed parsing SMILES 'CC6(C)CC4(CC(C)(C)c3cc2oc1c(C#N)c([g])c([t])c(C#N)c1oc2cc34)c5cc(O[d])c(O[e])cc56' for input: 'CC6(C)CC4(CC(C)(C)c3cc2oc

Количество валидных SMILES в датасете сорбции: 2275
Количество валидных SMILES в датасете диффузии: 2045

Обработка датасета сорбции...


[16:23:12] SMILES Parse Error: syntax error while parsing: CC6(C)CC4(CC(C)(C)c3cc2oc1c(C#N)c([g])c([t])c(C#N)c1oc2cc34)c5cc(O[d])c(O[e])cc56
[16:23:12] SMILES Parse Error: check for mistakes around position 36:
[16:23:12] (C)c3cc2oc1c(C#N)c([g])c([t])c(C#N)c1oc2c
[16:23:12] ~~~~~~~~~~~~~~~~~~~~^
[16:23:12] SMILES Parse Error: Failed parsing SMILES 'CC6(C)CC4(CC(C)(C)c3cc2oc1c(C#N)c([g])c([t])c(C#N)c1oc2cc34)c5cc(O[d])c(O[e])cc56' for input: 'CC6(C)CC4(CC(C)(C)c3cc2oc1c(C#N)c([g])c([t])c(C#N)c1oc2cc34)c5cc(O[d])c(O[e])cc56'
[16:23:12] SMILES Parse Error: syntax error while parsing: CC6(C)CC4(CC(C)(C)c3cc2oc1c(C#N)c([g])c([t])c(C#N)c1oc2cc34)c5cc(O[d])c(O[e])cc56
[16:23:12] SMILES Parse Error: check for mistakes around position 36:
[16:23:12] (C)c3cc2oc1c(C#N)c([g])c([t])c(C#N)c1oc2c
[16:23:12] ~~~~~~~~~~~~~~~~~~~~^
[16:23:12] SMILES Parse Error: Failed parsing SMILES 'CC6(C)CC4(CC(C)(C)c3cc2oc1c(C#N)c([g])c([t])c(C#N)c1oc2cc34)c5cc(O[d])c(O[e])cc56' for input: 'CC6(C)CC4(CC(C)(C)c3cc2oc


Обработка датасета диффузии...


[16:23:43] SMILES Parse Error: syntax error while parsing: CC6(C)CC4(CC(C)(C)c3cc2oc1c(C#N)c([g])c([t])c(C#N)c1oc2cc34)c5cc(O[d])c(O[e])cc56
[16:23:43] SMILES Parse Error: check for mistakes around position 36:
[16:23:43] (C)c3cc2oc1c(C#N)c([g])c([t])c(C#N)c1oc2c
[16:23:43] ~~~~~~~~~~~~~~~~~~~~^
[16:23:43] SMILES Parse Error: Failed parsing SMILES 'CC6(C)CC4(CC(C)(C)c3cc2oc1c(C#N)c([g])c([t])c(C#N)c1oc2cc34)c5cc(O[d])c(O[e])cc56' for input: 'CC6(C)CC4(CC(C)(C)c3cc2oc1c(C#N)c([g])c([t])c(C#N)c1oc2cc34)c5cc(O[d])c(O[e])cc56'
[16:23:43] SMILES Parse Error: syntax error while parsing: CC6(C)CC4(CC(C)(C)c3cc2oc1c(C#N)c([g])c([t])c(C#N)c1oc2cc34)c5cc(O[d])c(O[e])cc56
[16:23:43] SMILES Parse Error: check for mistakes around position 36:
[16:23:43] (C)c3cc2oc1c(C#N)c([g])c([t])c(C#N)c1oc2c
[16:23:43] ~~~~~~~~~~~~~~~~~~~~^
[16:23:43] SMILES Parse Error: Failed parsing SMILES 'CC6(C)CC4(CC(C)(C)c3cc2oc1c(C#N)c([g])c([t])c(C#N)c1oc2cc34)c5cc(O[d])c(O[e])cc56' for input: 'CC6(C)CC4(CC(C)(C)c3cc2oc

Расширение датасетов завершено!
